# Gets the metadata for a character from precache

In [302]:
import requests
import json
from bs4 import BeautifulSoup

In [303]:
def get_data(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.content, 'html.parser')
            # Find the win rate element by its class (this class name is subject to change)
            scripts = soup.find_all('script')
            for script in scripts:
                if script.string and '"header":' in script.string:
                    # Find the start of the JSON object
                    prefix = 'var precache = '
                    start = script.string.find(prefix) + len(prefix)

                    # Extract and parse the JSON string
                    json_str = script.string[start:-1].strip()
                    header_json = json.loads(json_str)
                    key = list(header_json.keys())
                    header_json[key[0]].update(header_json.pop(key[1]))
                    header_json = header_json[key[0]]
                    return header_json
            
            
        else:
            return f"Failed to retrieve data. Status code: {response.status_code}"
    except Exception as e:
        return f"An error occurred: {e}"
    


### Precalculate champion json data

In [304]:
def dict_to_json_file(json_data, file_name):
    try:
        # Open the file in write mode
        with open(file_name, 'w') as file:
            # Write the dictionary to the file in JSON format
            json.dump(json_data, file, indent=4)
        print(f"Dictionary successfully written to {file_name}")
    except Exception as e:
        print(f"An error occurred: {e}")

def json_file_to_dict(file_name):
    try:
        # Open the file for reading
        with open(file_name, 'r') as file:
            # Read and convert the JSON data into a Python dictionary
            data = json.load(file)
            return data
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

ids_json = json_file_to_dict('ids.json')
names_json = json_file_to_dict('names.json')

In [305]:
rounding = 2

def convert_champion_list(champion_json, to_convert):
    for key in to_convert:
        if key in champion_json:
            champion_json[key] = {names_json['champx'].get(str(lst[0]), str(lst[0])): {'wr':round(100*lst[2]/max(lst[1],1), rounding), 'w':lst[2] ,'n': lst[1]} for lst in champion_json[key]}
    return True

def convert_item_list(champion_json, to_convert):
    for key in to_convert:
        if key in champion_json:
            champion_json[key] = {names_json['item'].get(str(lst[0]), str(lst[0])): {'wr': lst[1], 'pr': lst[2], 'n': lst[3]} for lst in champion_json[key]}
    return True

def convert_stats_list(champion_json, to_convert):
    for key in to_convert:
        if key in champion_json:
            champion_json[key] = {lst[0]: lst[1:] for lst in champion_json[key]}
    return True

def convert_runes_list(champion_json, to_convert):
    for key in to_convert:
        if key in champion_json:
            champion_json[key] = champion_json[key]['stats']
            champion_json[key] = {names_json['rune'].get(key, key): value for key, value in champion_json[key].items()}

    return True

def scrub(champion_json, to_scrub):
    for key in to_scrub:
        if key in champion_json:
            del champion_json[key]
    return True
          
          


In [306]:
champions_all = [
    "aatrox", "ahri", "akali", "alistar", "amumu", "anivia", "annie", "aphelios",
    "ashe", "aurelionsol", "azir", "bard", "blitzcrank", "brand", "braum",
    "caitlyn", "camille", "cassiopeia", "chogath", "corki", "darius", "diana",
    "drmundo", "draven", "ekko", "elise", "evelynn", "ezreal", "fiddlesticks",
    "fiora", "fizz", "galio", "gangplank", "garen", "gnar", "gragas", "graves",
    "gwen", "hecarim", "heimerdinger", "illaoi", "irelia", "ivern", "janna",
    "jarvaniv", "jax", "jayce", "jhin", "jinx", "kaisa", "kalista", "karma",
    "karthus", "kassadin", "katarina", "kayle", "kayn", "kennen", "khazix",
    "kindred", "kled", "kogmaw", "leblanc", "leesin", "leona", "lillia",
    "lissandra", "lucian", "lulu", "lux", "malphite", "malzahar", "maokai",
    "masteryi", "missfortune", "mordekaiser", "morgana", "nami", "nasus",
    "nautilus", "neeko", "nidalee", "nocturne", "nunu", "olaf", "orianna",
    "ornn", "pantheon", "poppy", "pyke", "qiyana", "quinn", "rakan", "rammus",
    "reksai", "rell", "renata", "renekton", "rengar", "riven", "rumble", "ryze",
    "samira", "sejuani", "senna", "seraphine", "sett", "shaco", "shen", "shyvana",
    "singed", "sion", "sivir", "skarner", "smolder", "sona", "soraka", "swain", "sylas",
    "syndra", "tahmkench", "taliyah", "talon", "taric", "teemo", "thresh",
    "tristana", "trundle", "tryndamere", "twistedfate", "twitch", "udyr",
    "urgot", "varus", "vayne", "veigar", "velkoz", "vex", "vi", "viego",
    "viktor", "vladimir", "volibear", "warwick", "wukong", "xayah", "xerath",
    "xinzhao", "yasuo", "yone", "yorick", "yuumi", "zac", "zed", "ziggs",
    "zilean", "zoe", "zyra"
]

champions = ['smolder']


for champion in champions_all:
    url = f"https://lolalytics.com/lol/{champion}/build"
    champion_json = get_data(url)

    # Remove data we don't need
    to_scrub = ['key','cache','response','ts','earlySet','earlyItem','startSet','startItem','itemSets','spells','topStats','depth','graph','spell','timeWin','top','mythicItem','time','skills']
    scrub(champion_json, to_scrub)

    # Convert champion matchup/synergy data to something readable
    champion_winrate_convert = ['enemy_top', 'enemy_jungle', 'enemy_middle', 'enemy_bottom', 'enemy_support',
                                'team_top', 'team_jungle', 'team_middle', 'team_bottom', 'team_support']
    convert_champion_list(champion_json, champion_winrate_convert)

    # Convert items data to something readable
    item_winrate_convert = ['popularItem', 'item', 'item1', 'item2', 'item3', 'item4', 'item5', 'boots', 'winningItem']
    convert_item_list(champion_json, item_winrate_convert)

    # Convert stats data to dict
    convert_stats_list(champion_json, ['stats'])

    # Convert runes data to something readable
    convert_runes_list(champion_json, ['runes'])

    # Write it to the champion's json file
    dict_to_json_file(champion_json, f'Champion Data/{champion}.json')

Dictionary successfully written to Champion Data/aatrox.json
Dictionary successfully written to Champion Data/ahri.json
Dictionary successfully written to Champion Data/akali.json
Dictionary successfully written to Champion Data/alistar.json
Dictionary successfully written to Champion Data/amumu.json
Dictionary successfully written to Champion Data/anivia.json
Dictionary successfully written to Champion Data/annie.json
Dictionary successfully written to Champion Data/aphelios.json
Dictionary successfully written to Champion Data/ashe.json
Dictionary successfully written to Champion Data/aurelionsol.json
Dictionary successfully written to Champion Data/azir.json
Dictionary successfully written to Champion Data/bard.json
Dictionary successfully written to Champion Data/blitzcrank.json
Dictionary successfully written to Champion Data/brand.json
Dictionary successfully written to Champion Data/braum.json
Dictionary successfully written to Champion Data/caitlyn.json
Dictionary successfully 